In [1]:
import nltk
import itertools
import numpy as np
import csv
# sentence_start_token = 'start'
# sentence_end_token = 'end'
# unknown_token = 'unknown'
# text = 'Oh! my god, I dont know why did I buy this product, A waste of momney. This is outrageous, I dont get it. \
# My name is Sam Helsen and I like trekking. What is your name? Are you Sam Helsen?. I am going to America, the land of free.\
#  Last night I saw a bad man. Sam Helsen was caught smoking weed amid the crowd, this is indeed an outrageous attempt. The police charged fine, such a poor fellow. \
#  Sam has lost a lot of money in gambling and now he is loosing by paying fines. Sam bought a product yesterday, which was infact \
#  an expire product. He applied it yesterday to his face and got allergic itching. Such a unlucky fellow. He ones agin lost a lot of money \
#  treating the ailment. '   
data_path = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/reddit-comments-2015-08.csv"

vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
 
# Read the data and append SENTENCE_START and SENTENCE_END tokens
print ("Reading CSV file...")
with open(data_path, 'r') as f:
    reader = csv.reader(f, skipinitialspace=True)
    next(reader)
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print ("Parsed %d sentences." % (len(sentences)))
     

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
 
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print ("Found %d unique words tokens." % len(word_freq.items()))
 
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
 
print ("Using vocabulary size %d." % vocabulary_size)
print ("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
 
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    
print ("\nExample sentence: '%s'" % sentences[0])
print ("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])
 
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])


Reading CSV file...
Parsed 79170 sentences.
Found 65752 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'oven' and appeared 10 times.

Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'i', 'joined', 'a', 'new', 'league', 'this', 'year', 'and', 'they', 'have', 'different', 'scoring', 'rules', 'than', 'i', "'m", 'used', 'to', '.', 'SENTENCE_END']'


In [47]:
# Assign instance variables
word_dim = vocabulary_size
hidden_dim = 100
bptt_truncate = 4
# Randomly initialize the network parameters
U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

        

print (-np.sqrt(1./word_dim))
print ('')
print (U.shape)
print (U)
print (V.shape)
print (V)
print (W.shape)
print (W)

-0.0111803398875

(100, 8000)
[[ 0.00224304 -0.00030377  0.00417706 ...,  0.0026081   0.00424762
   0.00987281]
 [ 0.00376147  0.00515489  0.00457093 ...,  0.00030813 -0.00147325
   0.00851207]
 [ 0.00316309  0.00437324  0.0065804  ..., -0.00609013  0.00242317
   0.00520144]
 ..., 
 [-0.00824877  0.00055327 -0.00643888 ...,  0.00690315 -0.00103506
   0.01098181]
 [ 0.00227252 -0.00884253  0.00600094 ..., -0.00026247 -0.00115844
   0.00046335]
 [-0.00717248  0.00894428  0.00034896 ...,  0.00526743  0.00331113
  -0.0067716 ]]
(8000, 100)
[[ 0.08140482 -0.03362048  0.07209546 ...,  0.03852396  0.06284285
   0.04264014]
 [ 0.01305844 -0.05537182 -0.00351933 ..., -0.02110978  0.05523009
  -0.06193361]
 [ 0.06561785  0.05986148  0.00653587 ...,  0.06910809 -0.03686612
  -0.08767501]
 ..., 
 [-0.00398678 -0.01554461 -0.08671286 ..., -0.04349093 -0.02483941
   0.06279812]
 [ 0.06832471 -0.02821722 -0.09934568 ...,  0.06923698 -0.06767442
   0.09754821]
 [-0.05925875  0.09009937  0.02459723 ...

In [89]:
#x = X_train[10]
# print (x)
#T = len(x)
#s = np.zeros((T + 1, hidden_dim))
# print (T)
#s[-1] = np.zeros(hidden_dim)
#o = np.zeros((T, word_dim))


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# # For each time step...
# for t in np.arange(T):
#     #print (t)
# #     # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
    
#     print (s[t].shape, V.shape)
#     o[t] = softmax(V.dot(s[t]))
# print (s.shape)
# print (o.shape)

def forward_propagation(x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, hidden_dim))
    s[-1] = np.zeros(hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        print (s[t-1])
        s[t] = np.tanh(U[:,x[t]] + W.dot(s[t-1]))
        o[t] = softmax(V.dot(s[t]))
    return [o, s]

x = X_train[10]  
o, s = forward_propagation(x)
print (s.shape, o.shape)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[  2.24303323e-03   3.76145110e-03   3.16307744e-03   9.20302638e-03
   4.20667752e-03   9.47288487e-03   9.78740477e-03   8.35080233e-03
   6.15175532e-03   8.34451926e-03  -6.56946921e-03  -6.00393683e-03
  -7.77890198e-03  -1.02877747e-02   3.50829435e-03  -5.66671709e-03
  -6.08598368e-03  -1.02755253e-02  -4.64895149e-03  -6.70548231e-03
   3.35261312e-03   6.14540977e-03   5.33838053e-03  -7.43181320e-03
   5.69823986e-03  -1.95931085e-03  -1.95276459e-03   1.92418232e-03
  -5.68609832e-03  -1.49023999e-03  -3.86971996e-03  -5.67106909e-03
  -8.05714790e-03   1.06145696e-03  -2.31

In [71]:
def calculate_total_loss(x, y):
    L = 0
    # For each sentence...
#     print ('qaqqaqa ', len(y))
#     print ('ttttttt ', len(x))
    for i in np.arange(len(y)):
        #print ('ppopopopopopo ', x[i])
        o, s = forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L
 
def calculate_loss(x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return calculate_total_loss(x,y)/N
 
    
# For 1 examples
print ("Actual loss:",  calculate_loss([X_train[10]], [y_train[10]]))


Actual loss: 8.98725393291


In [73]:
# Doing it for first 100 training dataset
# Limit to 1000 examples to save time
print ("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print ("Actual loss: %f" % calculate_loss(X_train[:1000], y_train[:1000]))


Expected Loss for random predictions: 8.987197
Actual loss: 8.987092


In [90]:
def bptt(x, y):
    T = len(y)
    # Perform forward propagation
    o, s = forward_propagation(x)
    print (o.shape, s.shape)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(U.shape)
    dLdV = np.zeros(V.shape)
    dLdW = np.zeros(W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    print (delta_o.shape)
    # For each output backwards...
    # Backpropogation works from right to left, so we reverse the arrat, ex [0,1,2,3] be becomes [3,2,1,0]
    for t in np.arange(T)[::-1]:
        print ('')
        print (t)
        dLdV += np.outer(delta_o[t], s[t].T)
        print (delta_o[t].shape)
        print (s[t].T.shape)
        print (dLdV.shape)
        print (V.T.shape)
        print (delta_o[t].shape)
#         # Initial delta calculation
#         delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
#         # Backpropagation through time (for at most self.bptt_truncate steps)
#         for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
#             # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
#             dLdW += np.outer(delta_t, s[bptt_step-1])              
#             dLdU[:,x[bptt_step]] += delta_t
#             # Update delta for next step
#             delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
#     return [dLdU, dLdV, dLdW]

# We take a small example to 
bptt([0,1,2,3], [1,2,3,4]) # 0,1,2,3,4 is the word sequence for example:  0->I, 1->am, 2->going, 3->to, 4->hit, 5-> you 

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[  2.24303323e-03   3.76145110e-03   3.16307744e-03   9.20302638e-03
   4.20667752e-03   9.47288487e-03   9.78740477e-03   8.35080233e-03
   6.15175532e-03   8.34451926e-03  -6.56946921e-03  -6.00393683e-03
  -7.77890198e-03  -1.02877747e-02   3.50829435e-03  -5.66671709e-03
  -6.08598368e-03  -1.02755253e-02  -4.64895149e-03  -6.70548231e-03
   3.35261312e-03   6.14540977e-03   5.33838053e-03  -7.43181320e-03
   5.69823986e-03  -1.95931085e-03  -1.95276459e-03   1.92418232e-03
  -5.68609832e-03  -1.49023999e-03  -3.86971996e-03  -5.67106909e-03
  -8.05714790e-03   1.06145696e-03  -2.31

In [87]:
a = np.array([1,2,3,4])
b = np.array([10,11,12])
print (a.shape, b.shape)
print (np.outer(a,b))

(4,) (3,)
[[10 11 12]
 [20 22 24]
 [30 33 36]
 [40 44 48]]
